In [1]:
import pandas as pd
import seaborn as sns
import requests
from bs4 import BeautifulSoup as bs
import dfply
import re


### Функция для получения нужных страниц форума

In [2]:
def parse_forum(url, starts, headers):
    result = []
    for start in starts:
        request = requests.get(url = url + str(start), headers = headers)
        result.append(bs(request.text, 'html.parser'))
    return result

### Функция для спарсивания нужных кусков страницы  
(del нужно, потому что на каждой странице вверху есть опрос, который тоже спарсивается, но не нужен)

In [3]:
def parse_soups(soups, tag, class_, delete):
    result = []
    for el in soups:
        content = el.find_all(tag, class_ = class_)
        del content[delete]
        for elem in content:
            result.append(elem.text.strip())
    return result

###  Парсим все страницы форума  
starts содержит номер сообщения, с которого будет начинаться страница, на странице помещается 20 сообщений.  
Чтобы спарсить сообщения, которые появятся потом, нужно будет добавить нужные значения в список

In [4]:
starts = [0,20, 40, 60, 80, 100, 120, 140]
headers = {
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.5 Safari/605.1.15',
    'cookie': '_ga_W0JDW2V29Y=GS1.1.1661181837.2.1.1661184512.60.0.0; _ga=GA1.2.1025230771.1661155454; _gid=GA1.2.721157209.1661155454; style_cookie=null; phpbb3_alft2_k=; phpbb3_alft2_sid=fa5f9f17fa55d5a1a82d62c62159899a; phpbb3_alft2_u=1'
}

soups = parse_forum('https://forum.awd.ru/viewtopic.php?f=521&t=397637&start=', starts, headers)

### Парсим из страниц форума содержание и блок «автор», в котором содержится дата

In [5]:
contents = parse_soups(soups, 'div', 'content', 0)
authors = parse_soups(soups, 'p', 'author', 0)

Проверяем, все ли ок с длиной  
Если она соответствует количеству сообщений на форуме, то все ок

In [6]:
print(len(contents))
print(len(authors))

156
156


### Функция для парсинга конкретных данных по регулярному выражению  
Айди нужен, чтобы потом вручную прописать данные, которые не спарсятся, и не потерять порядок людей
hard = True нужен, потому что в сообщениях по июлю много «5.», он втупую несколько раз режет оставшееся

In [7]:
def get_data(regex, list, group_num = 2, hard = False, hard_regex = '(5\.)(.*)'):
    result = []
    for id, value in enumerate(list):
        try:
            reg = re.search(regex, value).group(group_num).strip()
            
            if hard == True:
                for i in range(3):
                    try:
                        reg = re.search(hard_regex, reg).group(2).strip()
                    except:
                        break
                        
                        
            result.append(reg)
        except:
            print("id = " + str(id) + "__Не пон " + str(value) + "\n___________")
            result.append(str(id) + " NaN")
    return result

### Парсим все нужные нам данные по регулярным выражениям  
  
Через del удаляем первые строки на каждой странице — в них содержится закрепленное сообщение модератора

In [ ]:
date = get_data('\d\d\s\w{2,4}\s\d\d\d\d', authors, 0)
city = get_data('(1\.)(.*?)(2\.)', contents)
visa_center = get_data('(2\.)(.*?)(3\.)', contents) 
raw_history = get_data('(4\.)(.*?)(5\.)', contents)
raw_limit = get_data('(5\.)(.+?)(6\.)', contents, hard = True)

del date[0]
del city[0]
del visa_center[0]
del raw_history[0]
del raw_limit[0]

In [ ]:
print(pd.Series(raw_history).to_string())

### Категоризуем данные визовой истории  
Хотел категоризовать на две группы — по длине виз и по странам, но по странам через тупую категоризацию не получилось, поэтому страны категоризуются вручную. На датасете из 150 строк это не сильно критично

In [9]:
history = []

for id, text in enumerate(raw_history):
    if (
        'нет' in text.lower()
        ):
        history.append("Нет")
        
    elif (
        'лет' in text.lower()
        ):
        history.append('Годовые и больше')
        
    elif (
        'пол' not in text.lower() and
        'год' in text.lower()
        ):
        history.append('Годовые и больше')
        
    elif (
        'пол' in text.lower()
        ):
        history.append('Полугодовые')
        
    elif (
        "мес" in text.lower()
        ):
        history.append('Меньше полугода')
        
    elif (
        'дне' in text.lower() or
        'кратко' in text.lower()
        ):
        history.append('Меньше полугода')
        
    elif (
        'поезд' in text.lower()
        ):
        history.append('Даты поездки')
        
    elif (
        'чист' in text.lower() or
        'никогда' in text.lower()
        ):
        history.append('Нет')
        
    else:
        history.append(str(id) +'_Не пон_' + text)
        
history

        



['Полугодовые',
 'Годовые и больше',
 'Годовые и больше',
 'Полугодовые',
 '4_Не пон_Не было виз',
 'Меньше полугода',
 'Годовые и больше',
 'Полугодовые',
 'Меньше полугода',
 'Годовые и больше',
 '10_Не пон_Не было виз',
 'Полугодовые',
 'Годовые и больше',
 'Полугодовые',
 '14_Не пон_08.2021 запросили доп. документы*, оправила, 0',
 'Годовые и больше',
 'Годовые и больше',
 'Годовые и больше',
 'Годовые и больше',
 'Годовые и больше',
 'Годовые и больше',
 'Полугодовые',
 'Полугодовые',
 'Полугодовые',
 'Годовые и больше',
 'Меньше полугода',
 'Годовые и больше',
 'Годовые и больше',
 'Годовые и больше',
 'Полугодовые',
 'Годовые и больше',
 'Годовые и больше',
 'Полугодовые',
 '33_Не пон_Не было раньше виз.',
 'Годовые и больше',
 'Годовые и больше',
 'Годовые и больше',
 'Годовые и больше',
 'Полугодовые',
 'Полугодовые',
 'Годовые и больше',
 'Годовые и больше',
 'Годовые и больше',
 'Годовые и больше',
 '44_Не пон_45 NaN',
 '45_Не пон_Шенген',
 'Полугодовые',
 'Полугодовые',
 'Г

### Категоризуем данные по полученным визам

In [ ]:
print(pd.Series(raw_limit).to_string())

In [10]:
limit = []

for id, text in enumerate(raw_limit):
    if (
        'отк' in text.lower()
        ):
        limit.append("Отказ")
        
    elif (
        'лет' in text.lower()
        ):
        limit.append('Год и больше')
        
    elif (
        'пол' not in text.lower() and
        'год' in text.lower()
        ):
        limit.append('Год и больше')
        
    elif (
        'пол' in text.lower()
        ):
        limit.append('Полгода')
        
    elif (
        "мес" in text.lower()
        ):
        limit.append('Меньше полугода')
        
    elif (
        'дне' in text.lower()
        ):
        limit.append('Меньше полугода')
        
    elif (
        'поезд' in text.lower()
        ):
        limit.append('Даты поездки')
        
    else:
        limit.append(str(id) +'_Не пон_' + text)
        
limit

        


['Год и больше',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Меньше полугода',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Меньше полугода',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Даты поездки',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Меньше полугода',
 'Отказ',
 'Год и больше',
 'Отказ',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Даты поездки',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Полгода',
 'Меньше полугода',
 'Год и больше',
 'Год и больше',
 'Полгода',
 'Год и больше',
 '44_Не пон_45 NaN',
 'Меньше полугода',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Отказ',
 'Год и больше',
 'Год и больше',
 'Меньше полугода',
 'Отказ',
 'Год и больше',
 'Год и больше',
 'Год и больше',
 'Год и больш

### Собираем датафрейм

In [12]:
df = pd.DataFrame()

df['date'] = date
df['city'] = city
df['visa_center'] = visa_center
df['history'] = history
df['raw_history'] = raw_history
df['limit'] = limit
df['raw_limit'] = raw_limit

df

,date,city,visa_center,history,raw_history,limit,raw_limit
0,28 янв 2021,Москва,ВЦ Визаметрик,Полугодовые,виза Германии на пол года,Год и больше,1 год
1,09 мар 2021,Екатеринбург,ВЦ Визаметрик,Годовые и больше,Годовые чешская и две немецкие,Год и больше,3 года
2,26 мар 2021,Москва,ВЦ Визаметрик,Годовые и больше,"Испания, 1 год",Год и больше,3 года
3,19 май 2021,Москва,Визаметрик,Полугодовые,"Чехия полгода, Германия год.",Год и больше,1 год
4,22 май 2021,Москва,Визаметрик,4_Не пон_Не было виз,Не было виз,Меньше полугода,3 месяца
...,...,...,...,...,...,...,...
150,151 NaN,Москва,ВЦ Visametric,Годовые и больше,"08 подала документы, 29.08 уведомление о готов...",Год и больше,"1 год, запрашивала также на год"
151,152 NaN,Москва,ВЦ Visametric,Годовые и больше,"Шенген (Германия) - 5 лет, 4 года (по сроку па...",Год и больше,5 лет (как просила)
152,153 NaN,Москва,Визаметрик,Годовые и больше,"3 испанских визы 6 мес, 1 и 2 года",Год и больше,"3 года, запрашивали 2"
153,154 NaN,Калининград,ВЦ,Годовые и больше,4 летняя немецкая туристическая,Год и больше,5 лет


Служебный блок, который нужен для коммитов или траблшутинга

In [31]:
!git commit -am 'Все нужные данные собраны, дальше ручной парсинг и анализ'
!git push

On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.ipynb_checkpoints/
	raw_df.csv

nothing added to commit but untracked files present (use "git add" to track)
To https://github.com/MustDie-green/AWD-forum-visa-parsing
 ! [rejected]        main -> main (fetch first)
error: failed to push some refs to 'https://github.com/MustDie-green/AWD-forum-visa-parsing'
hint: Updates were rejected because the remote contains work that you do
hint: not have locally. This is usually caused by another repository pushing
hint: to the same ref. You may want to first integrate the remote changes
hint: (e.g., 'git pull ...') before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.


In [33]:
!git fetch

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 681 bytes | 97.00 KiB/s, done.
From https://github.com/MustDie-green/AWD-forum-visa-parsing
   54926cc..a2eb2c8  main       -> origin/main


In [ ]:
r = requests.get('https://forum.awd.ru/viewtopic.php?f=521&t=397637&start=120', headers = headers)

sp = bs(r.text, 'html.parser')

sp.find_all('div', 'content')




In [ ]:
city = get_data('(1\.)(.*?)(2\.)', contents)
city

In [ ]:
visa_center = get_data('(2\.)(.*?)(3\.)', contents) 
visa_center

In [ ]:
date = get_data('\d\d\s\w{2,4}\s\d\d\d\d', authors, 0)
date

In [ ]:
visa_history = get_data('(4\.)(.*?)(5\.)', contents)
visa_history

In [ ]:
visa_limit = get_data('(5\.)(.+?)(6\.)', contents, hard = True)
visa_limit